In [22]:
pip install tokenizers

In [23]:
from pathlib import Path
import torch
import os
import pandas as pd
# the folder 'text' contains all the files
#paths = [str(x) for x in Path("./data/").glob("**/*.txt")]

from google.colab import drive
drive.mount('/content/drive')

# specify GPU
device = torch.device("cuda")

# Import from external file
X_train = pd.read_csv('/content/drive/My Drive/Data/output/x_train.csv')
X_test = pd.read_csv("/content/drive/My Drive/Data/output/x_test.csv")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [24]:
pip install transformers

In [25]:
import tensorflow as tf

from transformers import GPT2Config, TFGPT2LMHeadModel, GPT2Tokenizer
# loading tokenizer from the saved model path
#tokenizer = GPT2Tokenizer.from_pretrained(save_path)
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
'''
tokenizer.add_special_tokens({
  "eos_token": "</s>",
  "bos_token": "<s>",
  "unk_token": "<unk>",
  "pad_token": "<pad>",
  "mask_token": "<mask>"
})
'''
# creating the configurations from which the model can be made
config = GPT2Config(
  vocab_size=tokenizer.vocab_size,
  bos_token_id=tokenizer.bos_token_id,
  eos_token_id=tokenizer.eos_token_id
)
# creating the model
#model = TFGPT2LMHeadModel(config)
model = TFGPT2LMHeadModel.from_pretrained('gpt2')

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at gpt2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


In [26]:
single_string = ''
for i in range(len(X_train)):
  single_string += X_train['question'].iloc[i] + ' ' + tokenizer.bos_token + ' ' + X_train['answer'].iloc[i] + ' ' + tokenizer.eos_token + ' '
string_tokenized = tokenizer.encode(single_string)

Token indices sequence length is longer than the specified maximum sequence length for this model (410593 > 1024). Running this sequence through the model will result in indexing errors


In [27]:
'''
single_string = ''
for filename in paths:
  with open(filename, "r", encoding='utf-8') as f:
   x = f.read()
   single_string += x + tokenizer.eos_token
string_tokenized = tokenizer.encode(single_string)
'''

'\nsingle_string = \'\'\nfor filename in paths:\n  with open(filename, "r", encoding=\'utf-8\') as f:\n   x = f.read()\n   single_string += x + tokenizer.eos_token\nstring_tokenized = tokenizer.encode(single_string)\n'

In [28]:
#print(string_tokenized)

In [29]:
examples = []
block_size = 100
BATCH_SIZE = 12
BUFFER_SIZE = 1000
for i in range(0, len(string_tokenized) - block_size + 1, block_size):
  examples.append(string_tokenized[i:i + block_size])
#print(examples)
inputs, labels = [], []
for ex in examples:
  inputs.append(ex[:-1])
  labels.append(ex[1:])
  #print(labels)
dataset = tf.data.Dataset.from_tensor_slices((inputs, labels))
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

In [30]:
# defining our optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0)
# definining our loss function
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
# defining our metric which we want to observe
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
# compiling the model
model.compile(optimizer=optimizer, loss=[loss, *[None] * model.config.n_layer], metrics=[metric])

In [31]:
dataset

<BatchDataset shapes: ((12, 99), (12, 99)), types: (tf.int32, tf.int32)>

In [32]:
# defining our optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0)
# definining our loss function
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
# defining our metric which we want to observe
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
# compiling the model
model.compile(optimizer=optimizer, loss=[loss, *[None] * model.config.n_layer], metrics=[metric])

In [33]:
num_epoch = 10
history = model.fit(dataset, epochs=num_epoch)

Epoch 1/10


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:432: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


342/342 [==============================] - 104s 305ms/step - loss: 3.5338 - logits_loss: 3.5338 - logits_accuracy: 0.3504 - past_key_values_1_accuracy: 0.0022 - past_key_values_2_accuracy: 0.0020 - past_key_values_3_accuracy: 0.0015 - past_key_values_4_accuracy: 0.0045 - past_key_values_5_accuracy: 0.0023 - past_key_values_6_accuracy: 0.0020 - past_key_values_7_accuracy: 0.0014 - past_key_values_8_accuracy: 0.0021 - past_key_values_9_accuracy: 0.0012 - past_key_values_10_accuracy: 0.0015 - past_key_values_11_accuracy: 0.0011 - past_key_values_12_accuracy: 0.0016
Epoch 2/10
342/342 [==============================] - 104s 305ms/step - loss: 3.2618 - logits_loss: 3.2618 - logits_accuracy: 0.3777 - past_key_values_1_accuracy: 0.0022 - past_key_values_2_accuracy: 0.0020 - past_key_values_3_accuracy: 0.0015 - past_key_values_4_accuracy: 0.0045 - past_key_values_5_accuracy: 0.0023 - past_key_values_6_accuracy: 0.0020 - past_key_values_7_accuracy: 0.0013 - past_key_values_8_accuracy: 0.0020 - 

In [48]:
def predict(input_text):
  # encoding the input text
  input_text = input_text + ' ' + tokenizer.bos_token
  input_ids = tokenizer.encode(input_text, return_tensors='tf') 
  
  # getting out output
  beam_output = model.generate(
  input_ids,
  max_length = 1024,
  num_beams = 5,
  temperature = 0.7,
  no_repeat_ngram_size=2,
  num_return_sequences=5
)
  predicted_ans = tokenizer.decode(beam_output[0])
  predicted_ans = predicted_ans[predicted_ans.find(tokenizer.bos_token)+len(tokenizer.bos_token):]
  predicted_ans.replace(tokenizer.eos_token, '')
  predicted_ans.replace('\xa0', '')
  return predicted_ans

In [36]:
# Testing with custom test set
docs_new = ['What is coronavirus?',
            'What are symptoms of COVID',
            'What is COVID?',
            'I have a cough. Do I have corona?',
            'Does my friend have COVID?',
            'How many people in USA have COVID?',
            'How many people in the world have covid19?']


predicted_custom = []
for question in docs_new:
    predicted_custom.append(predict(question))

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


In [37]:
# View results
results_custom = pd.DataFrame(data = {'Question': docs_new, 'Predicted Answer': predicted_custom})
results_custom

,Question,Predicted Answer
0,What is coronavirus?,ndoftext|> Coronaviruses are a large family of...
1,What are symptoms of COVID,ndoftext|> I am a PhD student in the Departmen...
2,What is COVID?,ndoftext|> Coronavirus (COVID-19) is a new vir...
3,I have a cough. Do I have corona?,ndoftext|> Coronaviruses are a large family of...
4,Does my friend have COVID?,"ndoftext|> Yes, she has. \n\nHer symptoms are ..."
5,How many people in USA have COVID?,ndoftext|> The number of confirmed cases in th...
6,How many people in the world have covid19?,ndoftext|> There are currently no known cases ...


In [39]:
results_custom['Predicted Answer'].iloc[0]

'ndoftext|> Coronaviruses are a large family of viruses that cause a range of respiratory diseases including colds, flu, and severe acute respiratory syndrome (SARS-CoV-2).  \n\nThe virus that causes COVID-19 is called the "common cold", and is caused by the common cold adenovirus.  The virus causes about 80% of the world\'s respiratory infections, according to the World Organisation for Animal Health (OIE), and it is thought to be mainly transmitted through respiratory droplets produced when an infected person coughs or sneezes.\n\n\n  In the United States, the most common form of infection is respiratory syncytial fluid (RSD), which is a fluid-like fluid that is expelled from the upper respiratory tract when a respiratory infection occurs. This fluid is then expelled into the air, where it can be inhaled by people who are nearby or have close contact with someone who has the virus. However, there is no known way to precisely measure the amount of RSD in a person’s airways, or how muc

In [41]:
results_custom['Predicted Answer'].iloc[2]

'ndoftext|> Coronavirus (COVID-19) is a new virus that has not been previously identified in humans. It is thought to be caused by a virus called SARS-CoV-2.\n\nThe virus is believed to cause the following symptoms: fever, tiredness, and shortness of breath. These symptoms are usually mild and begin gradually. They may last for several days or may be worse than usual. Some people become infected but do not develop any symptoms. \n\n\n  The symptoms described by some patients are similar to those experienced by other coronaviruses, such as the common cold and the flu. However, they are different in severity and severity are not always consistent. For example, a person with a fever of 103.5°C and no symptoms for 14 days may develop a severe case of the virus. Other patients with similar symptoms may also develop pneumonia or other respiratory infections. The severity of these symptoms varies from person to person, depending on the underlying disease. In some cases, the severity may chang

In [42]:
results_custom['Predicted Answer'].iloc[4]

'ndoftext|> Yes, she has. \n\nHer symptoms are similar to those of other coronavirus patients, including fever, cough, and shortness of breath. Her doctor recommends that you take her to the emergency room, where she is being cared for, to be tested for the virus. She should be able to walk normally and is showing no signs of illness. However, her doctor is concerned that she may be acting abnormally and that her symptoms may not be the same as that of the flu, which she described as "very scary".  Her family is currently in the care of a specialist in Covid-19, who is treating her with carerty.com.au. Ms. Konyndyk said she had been in contact with a number of people who had similar symptoms, but none of them were as severe as she was.\nShe described how her mother had struggled with her anxiety and depression for years, before she developed the symptoms. "She was very self-centred and she didn\'t have any problems at all," she told CTV’s Your Morning on March 27, shortly after she sta

In [43]:
text = "i am interested in creating a web tool to predict the absolute risk of in-hospital death from a published risk model of covid-19 patients. is it possible to estimate the intercept from the significant, multivariable model coefficients published table 3 at http://doi.org/10.1016/s0140-6736 20 30566-3 answer "
predicted_ans = predict(text)

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


In [44]:
predicted_ans

' I am a PhD student in epidemiology and bioinformatics at the University of Waterloo. \n\nI am looking for a way to model the spread of a novel coronavirus (2019-nCoV) in a population of patients with severe acute respiratory illness (SARS) and their family members (L-R).  The model is based on the observation that the number of confirmed cases in the population is increasing every day, and that this increase in cases is due to a combination of factors:\n\n\nincreased hospitalisation,\nincreasing numbers of hospitalisations and deaths.\nThe model also assumes that patients are being cared for in isolation, so that they are not exposed to the virus in any way (e.g. they do not cough, sneeze, etc.).  In addition, the model assumes a constant infection rate of about 0.1%.  This means that if I were to extrapolate the exponential growth of the epidemic to all patients in my cohort, I would need to take into account the fact that there are many more patients than expected to die in this co

In [45]:
test = "no, for a few reasons. the most relevant is that, even if you could numerically determine the value of the intercept term in the model, you would not be able to tell what conditions are represented by the intercept term of the model fit by the researchers unless they provided some very specific details about their methods, e.g., did they center their predictors , etc. . the way these models are defined, you need to know what the reference point of each coefficient is. by convention the reference point for continuous variables is often zero, but the choice of reference is completely arbitrary for categorical variables though it does appear to be stated in the article . on a more conceptual note, models like this one are often limited in their ability to predict accurate class probabilities because their goal is to explain relationships between predictors, or to describe relative risk-reducing or risk-increasing relationships among the predictors in the model. often these models use only a limited number of variables to arrive at a parsimonious model, which tries to keep multicollinearity low, so that the individual effects of predictors are easier to interpret. another way of looking at it is this: the model may have been designed to address research questions such as: what is the relative importance of age with respect to the risk of an in-hospital death after controlling for other demographic factors . even if the model does this very well, it may be very bad at answering fundamentally different questions, such as what is the probability that a female will experience an in-hospital death"

In [46]:
import nltk
bleu_score =[]
bleu_score.append(nltk.translate.bleu_score.sentence_bleu(test, predicted_ans))

/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


In [47]:
bleu_score

[0.3053915492038505]

In [ ]:
predicted_answer = []
for question in x_test['question'].iloc[:100]:
  predicted_answer.append(predict(question))

In [ ]:
Dataframe(predicted_answer).to_csv('x_test_1.csv')